In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score



## Graded

### Load the data and split it into three sets, training, validation, and test, made up of 70%, 20%, and 10% of the observations.

In [3]:
# Load the data
data_url = 'https://raw.githubusercontent.com/dsahota-applied-data-analysis/data/main/spambase/spambase.data'
data = pd.read_csv(data_url, sep=',', header=None) # Read-in data with pandas

names_url = 'https://raw.githubusercontent.com/dsahota-applied-data-analysis/data/main/spambase/spambase.names'
names = pd.read_csv(names_url, sep='\t')



In [4]:
data

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [5]:
# Slpit train data
#X = data.iloc[:,:-1] # could also use this but would need to flatten y
X = data.drop(57, axis=1)
y = data[57]

In [6]:
# Split the data into training (70%), validation (20%), and test (10%)
X_train, temp_data_X = train_test_split(X, test_size=0.3, random_state=42)  # Split into 70% train, 30% test
y_train, temp_data_y = train_test_split(y, test_size=0.3, random_state=42)

X_val, X_test = train_test_split(temp_data_X, test_size=1/3, random_state=42)  # Split test into 2/3 validation, 1/3 real test
y_val, y_test = train_test_split(temp_data_y, test_size=1/3, random_state=42)

# Sanity check for the splits
print("Training Set:", X_train.shape)
print("Validation Set:", X_val.shape)
print("Test Set:", X_test.shape)

Training Set: (3220, 57)
Validation Set: (920, 57)
Test Set: (461, 57)


### **b**. Identify at least 5 neural network structures you want to try (number of hidden layers, number of nodes per layer).

**Baseline Model:**

##### Number of hidden layers: 1
##### Nodes in each layer: 10
##### This simple model will have just one hidden layer. This will see if having multiple layers will make the model more robust.

**Two Hidden Layers:**

##### Number of hidden layers: 2
##### Nodes in each layer: (64,32)
##### This model has two layers of size 64 and 32.


**Three Hidden Layers:**

##### Number of hidden layers: 3
##### Nodes in each layer: (128,64,32)
##### This model has three layers with an increased number of nodes.

**Four Hidden Layers:**

##### Number of hidden layers: 4
##### Nodes in each layer: (256,128,64,32)
##### This model will have 4 layers, the most I will test. The size of the nodes decreases with each later (as the network deepens)

**Even-sized Hidden Layers:**

##### Number of hidden layers: 2
##### Nodes in each layer: 100,100
##### This model serves to see how the size of the hidden layers impacts performance of the model

### **c**. Using the 70% training set, for each of these structures, identify the hyper-parameters (epochs and λ)that perform the best out-of-sample. You can split the training data into a sub-train/sub-test (say 50/20) or use cross-validation.

I will use the grid approach for finding the optimal hyperparameters. This uses scikitlearn's GridSearchCV:

[Towards Data Science article](https://towardsdatascience.com/hyperparameter-tuning-in-python-21a76794a1f7)

[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

Hyperparameters of interest: Learning rate and epoch size.

I will test learning rates of: [0.0001, 0.001, 0.01, 0.1]

 And iteration sixe of: [200, 300, 500]

In [7]:
sub_X_train, sub_X_test = train_test_split(X_train, test_size=0.3, random_state=42)
sub_y_train, sub_y_test = train_test_split(y_train, test_size=0.3, random_state=42)

# Create an MLPClassifier model
mlp = MLPClassifier()

Model 1: Baseline (1 hidden layer)

In [10]:
# Define the parameter grid for the models
param_grid_1 = {
    'hidden_layer_sizes': (10,),
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Learning rate
    'max_iter': [200, 300, 500],  # Number of epochs
    'random_state': [42]
}

# Set up GridSearchCV
clf_1 = GridSearchCV(mlp, param_grid_1, cv=5, scoring='accuracy')

In [11]:
# Fit the model
clf_1.fit(sub_X_train, sub_y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'hidden_layer_sizes': (10,),
                         'max_iter': [200, 300, 500], 'random_state': [42]},
             scoring='accuracy')

In [12]:
# Get best parameters
print("Best parameters for 1 hidden layer:")
print(clf_1.best_params_)

Best parameters for 1 hidden layer:
{'alpha': 0.01, 'hidden_layer_sizes': 10, 'max_iter': 200, 'random_state': 42}


Model 2: Two hidden

In [13]:
param_grid_2 = {
    'hidden_layer_sizes': (64, 32),
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Learning rate
    'max_iter': [200, 300, 500],  # Number of epochs
    'random_state': [42]
}
clf_2 = GridSearchCV(mlp, param_grid_2, cv=5, scoring='accuracy')


In [14]:
clf_2.fit(sub_X_train, sub_y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'hidden_layer_sizes': (64, 32),
                         'max_iter': [200, 300, 500], 'random_state': [42]},
             scoring='accuracy')

In [15]:
# Get best parameters
print("Best parameters for two hidden layers:")
print(clf_2.best_params_)

Best parameters for two hidden layers:
{'alpha': 0.01, 'hidden_layer_sizes': 32, 'max_iter': 200, 'random_state': 42}


Model 3: Three hidden layers

In [16]:
param_grid_3 = {
    'hidden_layer_sizes': (128, 64, 32),
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'max_iter': [200, 300, 500],
    'random_state': [42]
}
clf_3 = GridSearchCV(mlp, param_grid_3, cv=5, scoring='accuracy')



In [17]:
clf_3.fit(sub_X_train, sub_y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'hidden_layer_sizes': (128, 64, 32),
                         'max_iter': [200, 300, 500], 'random_state': [42]},
             scoring='accuracy')

In [18]:
# Get best parameters
print("Best parameters for three hidden layers:")
print(clf_3.best_params_)

Best parameters for three hidden layers:
{'alpha': 0.001, 'hidden_layer_sizes': 128, 'max_iter': 200, 'random_state': 42}


Model 4: Four hidden layers

In [19]:

param_grid_4 = {
    'hidden_layer_sizes': (256, 128, 64, 32),
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Learning Rate
    'max_iter': [200, 300, 500],  # Number of epochs
    'random_state': [42]
}
clf_4 = GridSearchCV(mlp, param_grid_4, cv=5, scoring='accuracy')


In [20]:
clf_4.fit(sub_X_train, sub_y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'hidden_layer_sizes': (256, 128, 64, 32),
                         'max_iter': [200, 300, 500], 'random_state': [42]},
             scoring='accuracy')

In [21]:
# Get best parameters
print("Best parameters for four hidden layers:")
print(clf_4.best_params_)

Best parameters for four hidden layers:
{'alpha': 0.001, 'hidden_layer_sizes': 128, 'max_iter': 200, 'random_state': 42}


Model 5: Two equal-size hidden layers

In [22]:

param_grid_5 = {
    'hidden_layer_sizes': (100, 100),
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Regularization strength
    'max_iter': [200, 300, 500],  # Number of epochs
    'random_state': [42]
}
clf_5 = GridSearchCV(mlp, param_grid_5, cv=5, scoring='accuracy')

In [23]:
clf_5.fit(sub_X_train, sub_y_train)

GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'hidden_layer_sizes': (100, 100),
                         'max_iter': [200, 300, 500], 'random_state': [42]},
             scoring='accuracy')

In [24]:
# Get best parameters
print("Best parameters for 2, equal hidden layers:")
print(clf_5.best_params_)

Best parameters for 2, equal hidden layers:
{'alpha': 0.0001, 'hidden_layer_sizes': 100, 'max_iter': 200, 'random_state': 42}


Conclusion: In all cases, the best number of epochs was 200, but the learning rate differed between the models.

### d. For each neural network structure, train it on the full 70% of data with the best hyper-parameters (epochs and λ) for that structure

In [31]:
best_params = [
    {'alpha': 0.01, 'hidden_layer_sizes': (10,), 'max_iter': 200, 'random_state': 42},
    {'alpha': 0.01, 'hidden_layer_sizes': (64, 32), 'max_iter': 200, 'random_state': 42},
    {'alpha': 0.001, 'hidden_layer_sizes': (128, 64, 32), 'max_iter': 200, 'random_state': 42},
    {'alpha': 0.001, 'hidden_layer_sizes': (256, 128, 64, 32), 'max_iter': 200, 'random_state': 42},
    {'alpha': 0.0001, 'hidden_layer_sizes': (100, 100), 'max_iter': 200, 'random_state': 42},
]


In [32]:
# List to store models (to reduce repetition)
models = []

# Train the five models with the best parameters
# Reference: class notes
for params in best_params:
    mlp = MLPClassifier(hidden_layer_sizes=params['hidden_layer_sizes'],
                        alpha=params['alpha'],
                        max_iter=params['max_iter'],
                        random_state=42)
    mlp.fit(X_train, y_train)
    models.append(mlp)

    print(f"Model with layers {params['hidden_layer_sizes']}:")
    print("Training set score: %f" % mlp.score(X_train, y_train))


MLPClassifier(alpha=0.01, hidden_layer_sizes=(10,), random_state=42)

Model with layers (10,):
Training set score: 0.927019


MLPClassifier(alpha=0.01, hidden_layer_sizes=(64, 32), random_state=42)

Model with layers (64, 32):
Training set score: 0.891304


MLPClassifier(alpha=0.001, hidden_layer_sizes=(128, 64, 32), random_state=42)

Model with layers (128, 64, 32):
Training set score: 0.892547


MLPClassifier(alpha=0.001, hidden_layer_sizes=(256, 128, 64, 32),
              random_state=42)

Model with layers (256, 128, 64, 32):
Training set score: 0.936646


MLPClassifier(hidden_layer_sizes=(100, 100), random_state=42)

Model with layers (100, 100):
Training set score: 0.924845


### e. Compare the out-of-sample performance of each structure, by evaluating the error rate on the 20% validation set. Identify the structure that performs the best

In [33]:

error_rates = []

for i, model in enumerate(models):
    y_pred = model.predict(X_val) # For validation set

    accuracy = accuracy_score(y_val, y_pred)

    # Calculate the error rate
    error_rate = 1 - accuracy
    error_rates.append(error_rate)

    print(f"Model {i+1} with layers {model.hidden_layer_sizes}: Error Rate = {error_rate:.4f}")


Model 1 with layers (10,): Error Rate = 0.0750
Model 2 with layers (64, 32): Error Rate = 0.1033
Model 3 with layers (128, 64, 32): Error Rate = 0.1065
Model 4 with layers (256, 128, 64, 32): Error Rate = 0.0739
Model 5 with layers (100, 100): Error Rate = 0.0815


In [34]:
# Get lowest error rate
# Reference: Pandas documentation
# https://pandas.pydata.org/docs/reference/api/pandas.Index.min.html

best_model_idx = error_rates.index(min(error_rates))
best_model = models[best_model_idx]
print(f"The best model is Model {best_model_idx+1} with structure {best_model.hidden_layer_sizes}, Error Rate = {min(error_rates):.4f}")



The best model is Model 4 with structure (256, 128, 64, 32), Error Rate = 0.0739


The structure that performed best was the model wth four hidden layers. It had an error rate of 0.0739

### f. For the structure that performs the best, using its best hyperparameters, train that neural network using 90% of the data (the training and validation sets).

In [35]:

# Combine the training and validation sets
X_train_val = pd.concat([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

# Train the model on combined training and validation
best_model.fit(X_train_val, y_train_val)

# Evaluate accuracy
y_train_val_pred = best_model.predict(X_train_val)
accuracy_train_val = accuracy_score(y_train_val, y_train_val_pred)
print(f"Accuracy on training and validation: {accuracy_train_val:.4f}")


MLPClassifier(alpha=0.001, hidden_layer_sizes=(256, 128, 64, 32),
              random_state=42)

Accuracy on training and validation: 0.9309


### g. Finally, evaluate your model by reporting its performance on the 10% test set.


In [36]:
from sklearn.metrics import f1_score

# get prediction on test set
y_test_pred = best_model.predict(X_test)

# Calculate various performance metrics
accuracy = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

# Print the performance metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.9393
F1 Score: 0.9218


Conclusion: I found the best model to be the one with the model hidden layers (4), where the size of the layers was the deepest out of the other models. The model performed best with a learning rate of 0.001 in 200 epochs. The final accuracy on the test set was 93.93%, and the F1 score was 0.9218, meaning the model had a strong performance.